## Preambule

In [1]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import json

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Parameter

In [2]:
hist_startyear = 1990

## Paths

In [3]:
path_main = Path("X:/user/dekkerm/Projects/ECEMF_T5.2/")
path_data = path_main / "Data" / str(hist_startyear)
path_figs = path_main / "Figures"

## Read data files

In [4]:
xr_total = xr.open_dataset(path_data / "xr_total.nc")
all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)
only_regions_iso = ['CVF', 'G20', "EU", "G7", "NA", "AU", "AF", "SIDS", "LDC", "WORLD"]
all_categories = ['C1+C2', 'C3', 'C6', 'C7']
only_regions_names = ['Climate Vulnerability Forum','G20',"European Union","G7","Northern America","Australasia","African Group","Small Island Developing States","Least Developed Countries"]

In [5]:
xr_budgets_static = xr.open_dataset(path_data / "xr_budgets_static.nc")
xr_budgets_linear = xr.open_dataset(path_data / "xr_budgets_linear.nc")
xr_budgets_scenario = xr.open_dataset(path_data / "xr_budgets_scenario.nc")
df_michel = pd.read_excel(Path("X:/user/dekkerm/Data/") / "NDC" / "NDC_comparison.xlsx", sheet_name="Michel")

## Preprocess

In [6]:
R = [["GF", "Grandfathering", "navy"],
     ["PC", "Per Capita (immediate)", "forestgreen"],
     ["PCC", "Per Capita Convergence", "red"],
     ["A1_gdp_net", "Approach 1 - GDP (net)", "orange"],
     ["A1_gdp_neg", "Approach 1 - GDP (neg)", "orange"],
     ["A1_gdp_pos", "Approach 1 - GDP (pos)", "orange"],
     ["A1_hdi_net", "Approach 1 - HDI (net)", "purple"],
     ["A1_hdi_neg", "Approach 1 - HDI (neg)", "purple"],
     ["A1_hdi_pos", "Approach 1 - HDI (pos)", "purple"],
     ["A2_gdp_net", "Approach 2 - GDP (net)", "silver"],
     ["A2_gdp_neg", "Approach 2 - GDP (neg)", "silver"],
     ["A2_gdp_pos", "Approach 2 - GDP (pos)", "silver"],
     ["A2", "Approach 2 - transition", "goldenrod"],
     ]
R = np.array(R).T
colors = R[2]
rules = R[0]
rulenames = R[1]

In [7]:
ar_gig = np.zeros(shape=(len(all_categories), len(rules)+5, len(all_regions_iso)))+np.nan

for c in tqdm(range(len(all_categories))):
    ccat = all_categories[c]
    for cty_i, cty in enumerate(all_regions_iso):
        ar_gig[c, 0, cty_i] = float(xr_total.sel(Time=2019, ISO=cty).GHG_p)

        # # Ratios
        # if total_kyoto_2020 > 0:
        #     ratio = total_co2_2020/total_kyoto_2020
        #     if cty == "CHN":
        #         ratio = 1
        #     ar_gig[c, 1, cty_i] = ratio
        
        # Curpol
        df = df_michel[df_michel.ISO == cty]['curpol_2030']
        if len(df) > 0:
            em_curpol = float(df_michel[df_michel.ISO == cty]['curpol_2030'])/1e3
            ar_gig[c, 2, cty_i] = em_curpol

        # NDC
        ndc_malte = [float(xr_total.sel(ISO=cty, Time=2030).NDC_l), float(xr_total.sel(ISO=cty, Time=2030).NDC_h)]
        ar_gig[c, 3, cty_i] = np.nanmean(ndc_malte)
        
        # Effort sharing rules
        for es_i, es in enumerate(["GF", "PC", "PCC", "A1_gdp_net", "A1_gdp_neg", "A1_gdp_pos", "A1_hdi_net", "A1_hdi_neg", "A1_hdi_pos", "A2_gdp_net", "A2_gdp_neg", "A2_gdp_pos", "A2_trans"]):
            try:
                ar_gig[c, es_i+4, cty_i] = float(xr_budgets_scenario.sel(Region=cty, Time=2030, Ccat=ccat, Variable=es).Value)
            except:
                ar_gig[c, es_i+4,cty_i] = np.nan
        
        # Scenarios
        # spendsw = []
        # for c in range(8):
        #     spent = float(xr_total.sel(Category="C"+str(c+1), Time=2030).GHG_f)
        #     spendsw.append(spent)
        # spendsw = np.array(spendsw)
        # spends = np.zeros(8)
        # for k in range(8):
        #     try:
        #         spends[k] = float(xr_ar6_iso3.sel(Region=cty, Variable='Emissions|Kyoto Gases|w/o LULUCF', Category="C"+str(k+1), Time=2030).Mean/1e3)
        #     except:
        #         spends[k] = np.nan
        # spends_new = fill_scenemissions(spends, spendsw)
        ar_gig[c, len(rules)+4, cty_i] = np.nan#float(xr_total.sel(Time=2030, ISO=cty, Category=ccat).GHG_f)
xr_preprocess = xr.DataArray(data=ar_gig*1e3,
                            dims=["Ccat", "Variable", "Region"],
                            coords=dict(Ccat=all_categories,
                                        Variable=["Emissions", "Ratio", "CurPol", "NDC", "GF", "PC", "PCC", "A1_gdp_net", "A1_gdp_neg", "A1_gdp_pos", "A1_hdi_net", "A1_hdi_neg", "A1_hdi_pos", "A2_gdp_net", "A2_gdp_neg", "A2_gdp_pos", "A2_trans", "Scenario"],
                                        Region=all_regions_iso))

  0%|          | 0/4 [00:00<?, ?it/s]C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_8164\11837393.py:23: RuntimeWarning: Mean of empty slice
  ar_gig[c, 3, cty_i] = np.nanmean(ndc_malte)
100%|██████████| 4/4 [00:16<00:00,  4.03s/it]


In [8]:
cs = np.zeros(shape=(13, len(all_countries_iso)))
for r_i, r in enumerate(all_countries_iso):
    ndc = float(xr_preprocess.sel(Variable="NDC", Region=r, Ccat="C1+C2"))
    for es_i, es in enumerate(["GF", "PC", "PCC", "A1_gdp_net", "A1_gdp_neg", "A1_gdp_pos", "A1_hdi_net", "A1_hdi_neg", "A1_hdi_pos", "A2_gdp_net", "A2_gdp_neg", "A2_gdp_pos", "A2_trans"]):
        ess = np.array(xr_preprocess.sel(Variable=es, Region=r)).astype(float)
        if not np.isnan(ndc):
            error = ess-ndc
            wh = np.where(error >= 0)[0]
            if len(wh) > 0:
                cs[es_i, r_i] = int(wh[0])
            else:
                cs[es_i, r_i] = int(3)
        else:
            cs[es_i, r_i] = np.nan
dicty = {}
dicty["Region"] = all_countries_iso
dicty["C_GF"] = cs[0]
dicty["C_PC"] = cs[1]
dicty["C_PCC"] = cs[2]
dicty["C_a1g_net"] = cs[3]
dicty["C_a1g_neg"] = cs[4]
dicty["C_a1g_pos"] = cs[5]
dicty["C_a1h_net"] = cs[6]
dicty["C_a1h_neg"] = cs[7]
dicty["C_a1h_pos"] = cs[8]
dicty["C_a2g_net"] = cs[9]
dicty["C_a2g_neg"] = cs[10]
dicty["C_a2g_pos"] = cs[11]
dicty["C_a2_trans"] = cs[12]
df_ndc = pd.DataFrame(dicty)

cs = np.zeros(shape=(13, len(all_countries_iso)))
for r_i, r in enumerate(all_countries_iso):
    curpol = float(xr_preprocess.sel(Variable="CurPol", Region=r, Ccat="C1+C2"))
    for es_i, es in enumerate(["GF", "PC", "PCC", "A1_gdp_net", "A1_gdp_neg", "A1_gdp_pos", "A1_hdi_net", "A1_hdi_neg", "A1_hdi_pos", "A2_gdp_net", "A2_gdp_neg", "A2_gdp_pos", 'A2_trans']):
        ess = np.array(xr_preprocess.sel(Variable=es, Region=r)).astype(float)
        if not np.isnan(curpol):
            error = ess-curpol
            wh = np.where(error >= 0)[0]
            if len(wh) > 0:
                cs[es_i, r_i] = int(wh[0])
            else:
                cs[es_i, r_i] = int(7)
        else:
            cs[es_i, r_i] = np.nan
dicty = {}
dicty["Region"] = all_countries_iso
dicty["C_GF"] = cs[0]
dicty["C_PC"] = cs[1]
dicty["C_PCC"] = cs[2]
dicty["C_a1g_net"] = cs[3]
dicty["C_a1g_neg"] = cs[4]
dicty["C_a1g_pos"] = cs[5]
dicty["C_a1h_net"] = cs[6]
dicty["C_a1h_neg"] = cs[7]
dicty["C_a1h_pos"] = cs[8]
dicty["C_a2g_net"] = cs[9]
dicty["C_a2g_neg"] = cs[10]
dicty["C_a2g_pos"] = cs[11]
dicty["C_a2_trans"] = cs[12]
df_curpol = pd.DataFrame(dicty)

## Plot

In [9]:
gf = np.array([df_ndc.iloc[c]['C_GF'] for c in range(len(df_ndc))])
gf[np.isnan(gf)] = 3
gf = gf.astype(int)

a1g = np.array([df_ndc.iloc[c]['C_a1g_net'] for c in range(len(df_ndc))])
a1g[np.isnan(a1g)] = 3
a1g = a1g.astype(int)

a1h = np.array([df_ndc.iloc[c]['C_a1h_net'] for c in range(len(df_ndc))])
a1h[np.isnan(a1h)] = 3
a1h = a1h.astype(int)

a2g = np.array([df_ndc.iloc[c]['C_a2g_net'] for c in range(len(df_ndc))])
a2g[np.isnan(a2g)] = 3
a2g = a2g.astype(int)

a2t = np.array([df_ndc.iloc[c]['C_a2_trans'] for c in range(len(df_ndc))])
a2t[np.isnan(a2t)] = 3
a2t = a2t.astype(int)
approaches = [a1g, a1h, a2g, a2t]

In [10]:
fig = make_subplots(rows=2, cols=2, subplot_titles=('Approach 1 (GDP)', 'Approach 1 (HDI)', 'Approach 2', 'Approach 2 (transition from GF)'), 
                                                    specs=[[{'type': 'choropleth'}, {'type': 'choropleth'}],
                                                           [{'type': 'choropleth'}, {'type': 'choropleth'}]],
                                                    horizontal_spacing = 0, vertical_spacing=0.02)

colorbar_abs = dict(len=1, x=1, tickvals = [0, 1, 2, 3], ticktext = ['1', '10', '100', '1000'])
ts = ['Below 1.5 degrees', 'Well below 2 degrees', 'Below 3 degrees', 'Below 4 degrees']
colorbar_pc = dict(len=1, x=1)

for var_i in range(4):
    if var_i == 0:
        cm= True
    else:
        cm= False
    t = approaches[var_i]
    #t[t >= 4] = np.nan
    fig.add_trace(
        go.Choropleth(
        locations=all_countries_iso,
        z = t,
        locationmode = 'ISO-3',
        colorscale = 'RdYlGn_r',
        zmax = 3,
        zmin = 0,
        text = [cty+": "+ts[t[cty_i]] for cty_i, cty in enumerate(all_countries_iso)],
        hovertemplate  = '%{text}',
        name="",
        marker_line_color='white', 
        marker_line_width=0.3,
        colorbar=dict(len=1, x=1, tickvals = [0, 1, 2, 3], ticktext = [all_categories[c]+": "+ts[c] for c in range(len(all_categories))], title={"text":"C-category"}),
        showscale=cm
    ), [1, 1, 2, 2][var_i], [1, 2, 1, 2][var_i])
fig.update_geos(showocean=True, oceancolor="aliceblue")
fig.update_geos(showlakes=True, lakecolor="aliceblue")

# limite map scope to USA 'africa', 'asia', 'europe', 'north america', 'south america', 'usa', 'world'
fig.update_geos(visible=False,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                showcoastlines=False,)
#fig.update_traces(showscale=False)
fig.update_layout(height=1200, width=2100)

In [11]:
# fig = make_subplots(rows=1, cols=1, subplot_titles=('','(a) Comparison NDCs',
#                                                     '(b) Comparison Current Policies'), 
#                                                     specs=[[{'type': 'choropleth'}]],#, {'type': 'choropleth'}]],
#                                                     horizontal_spacing = 0.04, vertical_spacing=0.05)#, column_widths=[0.5, 0.5])#, shared_yaxes=True)

# colorbar_abs = dict(len=1, x=1, tickvals = [0, 1, 2, 3], ticktext = ['1', '10', '100', '1000'])
# colorbar_pc = dict(len=1, x=1)
# ts = ['<1.5 degrees', 'Well below 2 degrees', 'Below 3 degrees', 'Below 4 degrees']

# fig.add_trace(
#     go.Choropleth(
#     locations=df_ndc['Region'],
#     z = df_ndc["C_GF"],
#     locationmode = 'ISO-3',
#     colorscale = 'YlOrRd', 
#     zmax = 3,
#     zmin = 0,
#     text = [df_ndc.iloc[c]['Region']+": "+all_categories[gf[c]] for c in range(len(df_ndc))],
#     hovertemplate  = '%{text}',
#     name="",
#     colorbar=dict(len=1, x=1, tickvals = [0, 1, 2, 3], ticktext = [all_categories[c]+": "+ts[c] for c in range(len(all_categories))], title={"text":"C-category"})
# ), 1, 1)
# fig.update_geos(showocean=True, oceancolor="whitesmoke")
# fig.update_geos(showlakes=True, lakecolor="whitesmoke")

# # fig.add_trace(
# #     go.Choropleth(
# #     locations=df_curpol['Region'],
# #     z = df_curpol["C_GF"],
# #     locationmode = 'ISO-3',
# #     colorscale = 'YlOrRd', 
# #     zmax = 7,
# #     zmin = 0,
# #     text = [df_curpol.iloc[c]['Region']+": C"+str(df_curpol.iloc[c]['C_GF']+1) for c in range(len(df_curpol))],
# #     hovertemplate  = '%{text}',
# #     name="",
# #     showlegend=False,
# #     showscale=False
# #     #colorbar=dict(len=1, x=1, tickvals = [0, 1, 2, 3, 4, 5, 6, 7], ticktext = ["C"+str(c+1)+": "+ts[c] for c in range(8)], title={"text":"C-category"})
# # ), 1, 2)
# # fig.update_geos(showocean=True, oceancolor="whitesmoke")
# # fig.update_geos(showlakes=True, lakecolor="whitesmoke")

# fig.update_layout(
#     updatemenus=[
#         dict(
#             type="buttons",
#             direction="right",
#             x=0.7,
#             y=-0.05,
#             showactive=True,
#             buttons=list(
#                 [
#                     dict(
#                         label="Grandfathering",
#                         method="update",
#                         args=[{"z": [df_ndc['C_GF'].astype(float), df_curpol['C_GF'].astype(float)],
#                                "text": [[df_ndc.iloc[c]['Region']+": "+all_categories[gf[c]] for c in range(len(df_ndc))]],
#                                "zmax": [3, 3],
#                                "zmin": [0, 0],
#                                }],
#                     ),
#                     # dict(
#                     #     label="Per-capita",
#                     #     method="update",
#                     #     args=[{"z": [df_ndc['C_PC'].astype(float), df_curpol['C_PC'].astype(float)],
#                     #            "text": [[df_ndc.iloc[c]['Region']+": C"+str(df_ndc.iloc[c]['C_PC']+1) for c in range(len(df_ndc))],
#                     #                      [df_curpol.iloc[c]['Region']+": C"+str(df_curpol.iloc[c]['C_PC']+1) for c in range(len(df_curpol))]],
#                     #            "zmax": [7, 7],
#                     #            "zmin": [0, 0],
#                     #            }],
#                     # ),
#                     # dict(
#                     #     label="Per-capita Convergence",
#                     #     method="update",
#                     #     args=[{"z": [df_ndc['C_PCC'].astype(float), df_curpol['C_PCC'].astype(float)],
#                     #            "text": [[df_ndc.iloc[c]['Region']+": C"+str(df_ndc.iloc[c]['C_PCC']+1) for c in range(len(df_ndc))],
#                     #                      [df_curpol.iloc[c]['Region']+": C"+str(df_curpol.iloc[c]['C_PCC']+1) for c in range(len(df_curpol))]],
#                     #            "zmax": [7, 7],
#                     #            "zmin": [0, 0],
#                     #            }],
#                     # ),
#                     dict(
#                         label="Approach 1 - GDP",
#                         method="update",
#                         args=[{"z": [df_ndc['C_a1g_net'].astype(float), df_curpol['C_a1g_net'].astype(float)],
#                                "text": [[df_ndc.iloc[c]['Region']+": "+all_categories[a1g[c]] for c in range(len(df_ndc))]],
#                                "zmax": [3, 7],
#                                "zmin": [0, 0],
#                                }],
#                     ),
#                     dict(
#                         label="Approach 1 - HDI",
#                         method="update",
#                         args=[{"z": [df_ndc['C_a1h_net'].astype(float), df_curpol['C_a1h_net'].astype(float)],
#                                "text": [[df_ndc.iloc[c]['Region']+": "+all_categories[a1h[c]] for c in range(len(df_ndc))]],
#                                "zmax": [3, 7],
#                                "zmin": [0, 0],
#                                }],
#                     ),
#                     dict(
#                         label="Approach 2 - GDP",
#                         method="update",
#                         args=[{"z": [df_ndc['C_a2g_net'].astype(float), df_curpol['C_a2g_net'].astype(float)],
#                                "text": [[df_ndc.iloc[c]['Region']+": "+all_categories[a2g[c]] for c in range(len(df_ndc))]],
#                                "zmax": [3, 7],
#                                "zmin": [0, 0],
#                                }],
#                     ),
#                 ]
#             ),
#         )
#     ]
# )
# fig.update_layout(height=800, width=1500)
# fig.update_layout(title_text = "Evaluating to which temperatures respective NDCs (unconditional) correspond.")
# fig.show()
# fig_comparison_emissions_ndc = fig

In [12]:
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_NDCperf/Figure_NDC_yann.svg")
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_NDCperf/Figure_NDC_yann.png", scale=3)

import os
from datetime import date
try:
    os.remove('X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_NDCperf/Figure_NDC_yann.html')
except:
    3
def html_w(typ):
    return '<html> '+typ+' <p style="font-family: Arial">'

today = date.today()
td = today.strftime("%B %d, %Y")

with open('X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_NDCperf/Figure_NDC_yann.html', 'a') as f:
    #f.write(html_w('<h1>')+'Figure 1 - Electricity Generation</p></h1>')
    f.write(html_w('<body>')+'Version: '+td+'</p></body>')
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))